Start by importing `Magma`.

In [1]:
import magma as m

Define a function that constructs a full adder.

In [2]:
def fulladder(A, B, C):
    return A^B^C, A&B|B&C|C&A

Initialize the IceStick headers. `J1` will be used as inputs; `J3` will be used as outputs.

In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.J1[0].input().on()
icestick.J1[1].input().on()
icestick.J1[2].input().on()
icestick.J3[0].output().on()
icestick.J3[1].output().on();

import mantle lattice ice40
import mantle lattice mantle40


In [4]:
main = icestick.DefineMain()

s, c = fulladder(main.J1[0], main.J1[1], main.J1[2])
m.wire(s, main.J3[0])
m.wire(s, main.J3[1])

m.EndDefine()

In [5]:
m.compile('build/fulladder', main)

compiling XOr2
compiling And2
compiling Or2
compiling main


In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif fulladder.blif' fulladder.v
arachne-pnr -q -d 1k -o fulladder.txt -p fulladder.pcf fulladder.blif 
icepack fulladder.txt fulladder.bin

/Users/hanrahan/git/magmathon/notebooks/build


In [7]:
%%bash 
# iceprog build/fulladder.bin

Show verilog. FullAdder has been mapped to ice40 primitives.

In [8]:
with open("build/fulladder.v", "r") as full_adder_verilog:
    print(full_adder_verilog.read())

module XOr2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h6666)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hEEEE)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module main (input [2:0] J1, output [1:0] J3);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
wire  inst5_O;
wire  inst6_O;
XOr2 inst0 (.I({J1[1],J1[0]}), .O(inst0_O));
XOr2 inst1 (.I({J1[2],inst0_O}), .O(inst1_O));
And2 inst2 (.I({J1[1],J1[0]}), .O(inst2_O));
And2 inst3 (.I({J1[2],J1[1]}), .O(inst3_O));
Or2 inst4 (.I({inst3_O,inst2_O}), .O(inst4_O));
And2 inst5 (.I({J1[0],J1[2]}), .O(inst5_O));
Or2 inst6 (.I({inst5_O,inst4

Show physical constraints file. FPGA pins are connected to headers J1 and J2.

In [9]:
with open("build/fulladder.pcf", "r") as full_adder_verilog:
    print(full_adder_verilog.read())

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J3[1] 61
set_io J3[0] 62

